In [ ]:
import pandas as pd

In [ ]:
# 1 & 2 - Reading the csv file
df = pd.read_csv("lab1_data.csv", sep=";")
print(df)

In [ ]:
# 3 - DATA PREPARATION 

In [ ]:
# Removing white spaces 
dirty_df = df
dirty_df[["id", "name", "price", "currency", "created_at"]] = dirty_df[["id", "name", "price", "currency", "created_at"]].apply(lambda x: x.str.strip())
dirty_df 

In [ ]:
# Dates format
dirty_df["created_at"] = pd.to_datetime(dirty_df["created_at"], errors='coerce')
dirty_df

In [ ]:
# From INTEGER ---> FLOAT
dirty_df["price"] = pd.to_numeric(dirty_df["price"], errors="coerce")
dirty_df

# index 3 and 14: neg price
# index 13: free ---> 0.0

In [ ]:
# 4 - Flagging the Nan or Nat 
luxury = 5000
checked_columns = ["id", "name", "price", "currency", "created_at"]
dirty_df["is_flagged"] = dirty_df[checked_columns].isna().any(axis=1)|(dirty_df["price"] < 0)|(dirty_df["price"] > luxury)
dirty_df

In [ ]:
# 5 - Avvisa omöjliga värden 
rejected_condition = ( 
    (dirty_df["id"].isna()) | 
    (dirty_df["price"] < 0) | 
    (dirty_df)["price"].isna() |
    ( dirty_df["currency"].isna()) 
    )

rejected_df = dirty_df[rejected_condition].copy()
accepted_df = dirty_df[~rejected_condition].copy()

In [ ]:
rejected_df
# pdf 6 sida 52 reject när obligatoriskt fält saknas = currency 
# pris at 0.0 is not an issue as string was "free" and intend is clear

In [27]:
rejected_df.to_csv("rejected_products.csv", index=False)
print("rejected_products created!")

rejected_products created!


In [ ]:
accepted_df

In [24]:
# 6 - analytics_summary preparation with utlity methods
# From accepted_df
analytics = {
    "snitpris" : [accepted_df['price'].mean().round(2)],
    "medianpris" : [accepted_df['price'].median()],
    "antal_produkter" : [len(accepted_df)],
    "antal_produkter_inget_pris" : [rejected_df['price'].isna().sum()],
    }
analytics

{'snitpris': [np.float64(23455.59)],
 'medianpris': [np.float64(600.0)],
 'antal_produkter': [46],
 'antal_produkter_inget_pris': [np.int64(5)]}

In [28]:
# From dict ---> DataFrame
analytics_summary = pd.DataFrame(analytics)
analytics_summary.to_csv("analytics_summary.csv", index=False)
print("analytics_summary created!")

analytics_summary created!


In [ ]:
# price_analysis.csv - Top 10 dyraste produkter
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
top_10_expensive = pd.DataFrame(accepted_df).sort_values(by=["price"], ascending=False).head(10).copy()
top_10_expensive


In [ ]:
# Top 10 produkter med mest avvikande pris 
# After speaking with Kristoffer => 2 outliers versions: including flagged valued and excluding flagged values
# We will have 2 dataframes here

# --- outliers INCLUDING flagged values ---
outliers_all = accepted_df.copy()
median_price_all = outliers_all["price"].median()
outliers_all["deviation"] = (outliers_all["price"] - median_price_all).abs()
#outliers_all

top_10_outliers_all  =( outliers_all.sort_values(by="deviation", ascending=False).head(10).copy() )
top_10_outliers_all

In [ ]:
# --- outliers EXCLUDING flagged values ---
outliers_not_flagged_df = accepted_df[accepted_df["is_flagged"] == False].copy()
median_price_not_flagged = outliers_not_flagged_df["price"].median()
outliers_not_flagged_df["deviation"]=(outliers_not_flagged_df["price"] - median_price_not_flagged).abs()

top_10_outliers_not_flagged = ( outliers_not_flagged_df.sort_values(by="deviation", ascending=False).head(10).copy() )
top_10_outliers_not_flagged

In [ ]:
# Merging the 3 dataframes, top_10_expensive & the 2 outliers
# source https://www.geeksforgeeks.org/python/pandas-concat-function-in-python/
top_10_expensive["category"] = "top_10_expensive"
top_10_outliers_all["category"] = "outliers_including_flagged"
top_10_outliers_not_flagged["category"] = "outliers_excluding_flagged"

price_analyis = pd.concat([top_10_expensive, top_10_outliers_all, top_10_outliers_not_flagged], ignore_index=True)
price_analyis